### Generate test cases 10 times using different strategy

In [2]:
import subprocess
import os

encoding = 'A32'
arch = 'AArch32'
strategy = 'random-symbols'
if arch == 'AArch32':
    xml_file = '../../test-generator/mra_tools/v8.6/ISA_AArch32_xml_v86A-2019-12'
else:
    xml_file =  '../../test-generator/mra_tools/v8.6/ISA_A64_xml_v86A-2019-12'

# run the test case generator script
script_to_run = '../../test-generator/genInsts.py'
script_arguments = [
    '--altslicesyntax', 
    '--demangle', 
    '--verbose', 
    '-o', '../../test-generator/mra_tools/arch/arch', 
    xml_file, 
    '--encoding', encoding, 
    '--arch', arch, 
    '--strategy', strategy
]
try:
    if not os.path.exists(f"{strategy}/{encoding}"):
       os.makedirs(f"{strategy}/{encoding}")
    # Run the script using subprocess
    for i in range(10):
        subprocess.run(['python3', script_to_run] + script_arguments, check=True)
        subprocess.run(['mv', f'{encoding}.txt', f'{strategy}/{encoding}/{encoding}_{i}.txt'], check=True)
        subprocess.run(['mv', f'{encoding}_constraints.json', f'{strategy}/{encoding}/{encoding}_constraints_{i}.json'], check=True)

except subprocess.CalledProcessError as e:
    print(f"Error running {script_to_run}: {e}")


In [5]:
import json
encoding = 'T16'

def calculate_statistics(instspath, encoding):
    results = {
        "number": [],
        "cover_encoding": [],
        "cover_instruction": [],
        "cover_constraint": []
    }
    for i in range(1):
        valid_file = f"{instspath}/{encoding}_{i}.txt"
        with open(valid_file,"r") as f:
            lines = f.readlines()
            results["number"].append(len(lines))
        
        valid_constraint_file = f"{instspath}/{encoding}_constraints_{i}.json"
        tmp = set()
        with open(valid_constraint_file,"r") as f:
            constraint_file_data = json.load(f)
            results["cover_encoding"].append(len(constraint_file_data.keys()))
            results["cover_constraint"].append(str(constraint_file_data).count("True"))
            for k in constraint_file_data:
                tmp.add(k.split("#")[1])
        results["cover_instruction"].append(len(tmp))
    return results


# random = calculate_statistics(f"random/{encoding}", encoding)
random_symbols = calculate_statistics(f"random-symbols/{encoding}", encoding)
symbolic = calculate_statistics(f"symbolic/{encoding}", encoding)

# print(random)
print(random_symbols)
print(symbolic)

{'number': [886], 'cover_encoding': [78], 'cover_instruction': [68], 'cover_constraint': [107]}
{'number': [925], 'cover_encoding': [76], 'cover_instruction': [67], 'cover_constraint': [119]}


In [13]:
from scipy.stats import mannwhitneyu
from cliffs_delta import cliffs_delta
def compute_vda(statistic, n1, n2):
    return (2 * statistic) / (n1 * n2) - (n1 + n2 + 1) / (n1 * n2)

# Perform Mann-Whitney U test
statistic, p_value= mannwhitneyu(symbolic['cover_encoding'], random['cover_encoding'])
result =  cliffs_delta(symbolic['cover_encoding'], random['cover_encoding'])
print(result)
statistic, p_value = mannwhitneyu(symbolic['cover_instruction'], random['cover_instruction'])
result =  cliffs_delta(symbolic['cover_instruction'], random['cover_instruction'])
print(statistic, p_value)
print(result)
statistic, p_value = mannwhitneyu(symbolic['cover_constraint'], random['cover_constraint'])
result =  cliffs_delta(symbolic['cover_constraint'], random['cover_constraint'])
print(statistic, p_value)
print(result)

(1.0, 'large')
100.0 6.113336892015162e-05
(-1.0, 'large')
100.0 0.00012855089812896465
(1.0, 'large')
